# Quantized SNN for MNIST

## Imports

In [ ]:
# SNN
import snntorch as snn
from snntorch import surrogate
from snntorch import functional as SF
from snntorch.functional import quant
from snntorch import utils
from snntorch import spikeplot as splt
from snntorch import spikegen

# Quantization
import brevitas.nn as qnn

# Torch
import torch
from torch import nn
from torch.nn import Module
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Other
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

## 1. Setting up MNIST

In [ ]:
batch_size = 128
data_path  = './data/mnist'
num_class  = 10

### 1.1 Download Dataset

In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0,), (1,))
])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

# Make a sanity check plot
fig, axs = plt.subplots(1, 3, figsize=(9, 3))
for i in range(3):
    image, label = mnist_train[i]
    axs[i].imshow(image.squeeze(0).numpy(), cmap="gray")
    axs[i].set_title(f"Label: {label}")

### 1.2 Dataloader and Spike Encoding

In [ ]:
# Create dataloader
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

# --- The rest in this cell is just to show how to encode data into spikes ---

# Fetch a batch of data
data = iter(train_loader)
data_it, targets_it = next(data)

# Encode it into spikes
#   - linear, normalize, clip are IMPORTANT for the spikegen.latency function
#     details here: https://snntorch.readthedocs.io/en/latest/tutorials/tutorial_1.html
spike_data = spikegen.latency(data_it, num_steps=100, tau=5, threshold=0.01, linear=True, normalize=True, clip=True)

# Plot to keep me sane
fig = plt.figure(facecolor="w", figsize=(12, 3))
ax = fig.add_subplot(111)
splt.raster(spike_data[:, 0].view(100, -1), ax, s=25, c="black")
plt.title("Input Layer")
plt.xlabel("Time step")
plt.ylabel("Neuron Number")
plt.show()

### 1.3 Network Definition

In [ ]:
config = {
    "num_epochs": 5,       # Number of epochs to train for (per trial)
    "batch_size": 128,      # Batch size
    "seed": 0,              # Random seed
    
    # Quantization
    "num_bits": 4,          # Bit resolution
    
    # Network parameters
    "grad_clip": False,     # Whether or not to clip gradients
    "weight_clip": False,   # Whether or not to clip weights
    "batch_norm": True,     # Whether or not to use batch normalization
    "dropout": 0.07,        # Dropout rate
    "beta": 1.00,           # Decay rate parameter (beta)
    "threshold": 30,        # Threshold parameter (theta)
    "lr": 3.0e-3,           # Initial learning rate
    "slope": 5.6,           # Slope value (k)
    
    # Fixed params
    "num_steps": 100,       # Number of timesteps to encode input for
    "correct_rate": 0.8,    # Correct rate
    "incorrect_rate": 0.2,  # Incorrect rate
    "betas": (0.9, 0.999),  # Adam optimizer beta values
    "t_0": 4690,            # Initial frequency of the cosine annealing scheduler
    "eta_min": 0,           # Minimum learning rate
}

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

class Net(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_bits   = config["num_bits"]
        self.thr        = config["threshold"]
        self.slope      = config["slope"]
        self.beta       = config["beta"]
        self.num_steps  = config["num_steps"]
        self.batch_norm = config["batch_norm"]
        
        # Initialize Layers
        self.fc1        = qnn.QuantLinear(32*32, 10, bias=False, weight_bit_width=self.num_bits)
        self.lif1       = snn.Leaky(self.beta, threshold=self.thr)

    def forward(self, x):
        # Flatten the input
        # x = x.view(x.size(0), -1)
        
        # Initialize hidden states and outputs at t=0
        mem1 = self.lif1.init_leaky()
        
        # Record the final layer
        spk_rec = []
        mem_rec = []
        for step in range(self.num_steps):
            cur = self.fc1(x[step].view(x.shape[1], -1))
            spk1, mem1 = self.lif1(cur, mem1)

            spk_rec.append(spk1)
            mem_rec.append(mem1)
        
        return torch.stack(spk_rec), torch.stack(mem_rec)

net = Net(config).to(device)

### 1.4 Define Optimizer, LR Scheduler and Loss Function

In [ ]:
optimizer = torch.optim.Adam(
    net.parameters(),
    lr=config["lr"],
    betas=config["betas"]
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=config["t_0"],
    eta_min=config["eta_min"],
    last_epoch=-1
)

criterion = SF.mse_count_loss(
    correct_rate=config["correct_rate"],
    incorrect_rate=config["incorrect_rate"]
)

### 1.5 Training and Evaluation

In [ ]:
def train(config, net, trainloader, criterion, optimizer, device="cpu", scheduler=None):
    """
    Complete one epoch of training.
    """
    
    net.train()
    loss_accum = []
    lr_accum = []
    for data, labels in trainloader:
        data, labels = data.to(device), labels.to(device)
        # Encode data into spikes
        data = spikegen.latency(data, num_steps=100, tau=5, threshold=0.01, linear=True, normalize=True, clip=True)
        spk_rec, mem_rec = net(data)
        loss = criterion(spk_rec, labels)
        optimizer.zero_grad()
        loss.backward()

        ## Enable gradient clipping
        if config["grad_clip"]:
            nn.utils.clip_grad_norm_(net.parameters(), 1.0)

        ## Enable weight clipping
        if config["weight_clip"]:
            with torch.no_grad():
                for param in net.parameters():
                    param.clamp_(-1, 1)

        optimizer.step()
        scheduler.step()
        loss_accum.append(loss.item() / config["num_steps"])
        lr_accum.append(optimizer.param_groups[0]["lr"])

    return loss_accum, lr_accum

def test(config, net, testloader, device="cpu"):
    """
    Calculate accuracy on full test set.
    """

    correct = 0
    total = 0
    with torch.no_grad():
        net.eval()
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            images = spikegen.latency(images, num_steps=100, tau=5, threshold=0.01, linear=True, normalize=True, clip=True)
            outputs, _ = net(images)
            accuracy = SF.accuracy_rate(outputs, labels)
            total += labels.size(0)
            correct += accuracy * labels.size(0)

    return 100 * correct / total

loss_list = []
lr_list = []

## Load model instead of training
load_model = True
if load_model:
    net.load_state_dict(torch.load('models/mnist.pth'))
else:
    print(f"=======Training Network=======")
    for epoch in range(config['num_epochs']):
        loss, lr  = train(config, net, train_loader, criterion, optimizer, device, scheduler)
        loss_list = loss_list + loss
        lr_list   = lr_list + lr
        # Test
        test_accuracy = test(config, net, test_loader, device)
        print(f"Epoch: {epoch} \tTest Accuracy: {test_accuracy}")

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.plot(loss_list, color='tab:orange')
    ax2.plot(lr_list, color='tab:blue')
    ax1.set_xlabel('Iterationb')
    ax1.set_ylabel('Loss', color='tab:orange')
    ax2.set_ylabel('Learning Rate', color='tab:blue')
    plt.show()

In [ ]:
save_model = False
if save_model:
    dir = "./models"
    if not os.path.exists(dir):
        os.makedirs(dir)
    torch.save(net.state_dict(), f"{dir}/mnist.pth")

### 1.5 Network Results and Test

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(12, 6))
iter_test = iter(test_loader)
data_it, targets_it = next(iter_test)

dataset = test_loader.dataset
num_samples = len(dataset)
ran_idx = torch.randint(0, num_samples, (3,))

for i, idx in enumerate(ran_idx):
    # Get some data
    data_tmp, target_tmp = dataset[idx]

    # Spike encode it
    spike_data = spikegen.latency(data_tmp, num_steps=100, tau=5, threshold=0.01, linear=True, normalize=True, clip=True)
    spike_data = spike_data.to(device)

    # Forward pass
    spk_rec, mem_rec = net(spike_data)

    # Besværgelse (just summing the spikes)
    pred = torch.argmax(spk_rec.sum(dim=0).squeeze()).item()

    # Plot
    splt.raster(spk_rec[:, 0].view(100, -1), ax[i], s=25, c="black")
    ax[i].set_yticks(np.arange(0, 10, 1))
    ax[i].set_title(f"Prediction: {pred}, Target: {target_tmp}")

plt.subplots_adjust(hspace=0.5)

### 1.7 Extract Quantized Weights

In [ ]:
# This seems to work
scaled_weights      = net.fc1.quant_weight()/net.fc1.quant_weight().scale
scaled_weights      = scaled_weights.cpu().detach().numpy()
scaled_threshold    = net.lif1.threshold/net.fc1.quant_weight().scale
print(f"Scaled threshold: {scaled_threshold}")

for i in range(scaled_weights.shape[1]):
    if not np.all(scaled_weights[:, i] == 0):
        print(f"First non-zero input index found: {i}")
        print(f"Weights from input {i} to outputs: {scaled_weights[:, i]}")
        break

weights = net.fc1.quant_weight()/net.fc1.quant_weight().scale
weights = weights.cpu().detach().numpy()

qsyn_bin = []
for in_nrn in range(0, 1024):
    syn_weights = scaled_weights[:, in_nrn]
    for syn in syn_weights:
        qsyn = int(round(syn))
        qsyn = max(-8, min(7, qsyn))
        qsyn_bin.append(format(qsyn & 0b1111, '04b'))

str_var = ""
lines = []
for syn in qsyn_bin:
    str_var += syn
    if len(str_var) == 40:
        lines.append(str_var)
        str_var = ""

with open("weights.txt", "w") as f:
    for line in lines:
        f.write(line + "\n")

## Extract Spike Encoded Number for Testbench

In [ ]:
## Select a datapoint from the dataset
dataset = test_loader.dataset
data_tmp, target_tmp = dataset[6]

## Spike encode it
spike_data = spikegen.latency(data_tmp, num_steps=100, tau=5, threshold=0.01, linear=True, normalize=True, clip=True)
spike_data = spike_data.to(device)

## Forward pass
spk_rec, mem_rec = net(spike_data)
pred = torch.argmax(spk_rec.sum(dim=0).squeeze()).item()

fig, ax = plt.subplots(2, 1, figsize=(12, 6))
splt.raster(spike_data[:, 0].view(100, -1), ax[0], s=25, c="black")
splt.raster(spk_rec[:, 0].view(100, -1), ax[1], s=25, c="black")
ax[0].set_title(f"Prediction: {pred}, Target: {target_tmp}")

## Detach as we will use these for comparison with the testbench
spk_rec = spk_rec.cpu().detach().numpy()
mem_rec = mem_rec.cpu().detach().numpy()

In [ ]:
## Create array of timebinned events
exp_data = spike_data[:, 0, :, :]
exp_data = exp_data.view(exp_data.shape[0], -1)

events = [[] for _ in range(100)]

## Loop timesteps
for t in range(0, 100):
    t_data = exp_data[t, :]
    non_zero_indices = (t_data != 0).nonzero(as_tuple=True)[0]
    non_zero_values = t_data[non_zero_indices]
    
    events[t] = non_zero_indices.tolist()

events_arr = np.array(events, dtype=object)

## Read results from testbench
df = pd.read_csv('data/testbench/mnist_spk_rec.csv', names=['event_number', 'out_fifo_wdata'])
df['out_fifo_wdata'] = df['out_fifo_wdata'].apply(lambda b: int(f"{b}", 2))

x = np.arange(0, 100, 1)
nrn_0 = np.zeros_like(x)
nrn_1 = np.zeros_like(x)
nrn_2 = np.zeros_like(x)
nrn_3 = np.zeros_like(x)
nrn_4 = np.zeros_like(x)
nrn_5 = np.zeros_like(x)
nrn_6 = np.zeros_like(x)
nrn_7 = np.zeros_like(x)
nrn_8 = np.zeros_like(x)
nrn_9 = np.zeros_like(x)

event_counter = 0
event_counter_last = 0
for t in range(0, 100):
    t_events = events_arr[t]
    event_counter += len(t_events)

    for event, nrn in zip(df['event_number'], df['out_fifo_wdata']):
        nrn_idx = 9 - nrn
        if (event <= event_counter) and (event > event_counter_last):
            if nrn_idx == 0:
                nrn_0[t] = 1
            elif nrn_idx == 1:
                nrn_1[t] = 1
            elif nrn_idx == 2:
                nrn_2[t] = 1
            elif nrn_idx == 3:
                nrn_3[t] = 1
            elif nrn_idx == 4:
                nrn_4[t] = 1
            elif nrn_idx == 5:
                nrn_5[t] = 1
            elif nrn_idx == 6:
                nrn_6[t] = 1
            elif nrn_idx == 7:
                nrn_7[t] = 1
            elif nrn_idx == 8:
                nrn_8[t] = 1
            elif nrn_idx == 9:
                nrn_9[t] = 1
    event_counter_last += len(t_events)

fig, ax = plt.subplots(1, 1, figsize=(12, 4))
ax.plot(x, nrn_4, linewidth=2, color='blue', alpha=0.5, label="FENRIR")
ax.plot(x, spk_rec[:, 0, 4], linewidth=2, color='red', alpha=0.5, label="snntorch")

ax.legend()
ax.set_xlim(0, 100)
ax.set_title("nrn_4")

print("spk_rec_0 sum:", np.sum(spk_rec[:, 0, 0]))
print("spk_rec_1 sum:", np.sum(spk_rec[:, 0, 1]))
print("spk_rec_2 sum:", np.sum(spk_rec[:, 0, 2]))
print("spk_rec_3 sum:", np.sum(spk_rec[:, 0, 3]))
print("spk_rec_4 sum:", np.sum(spk_rec[:, 0, 4]))
print("spk_rec_5 sum:", np.sum(spk_rec[:, 0, 5]))
print("spk_rec_6 sum:", np.sum(spk_rec[:, 0, 6]))
print("spk_rec_7 sum:", np.sum(spk_rec[:, 0, 7]))
print("spk_rec_8 sum:", np.sum(spk_rec[:, 0, 8]))
print("spk_rec_9 sum:", np.sum(spk_rec[:, 0, 9]))

print("nrn_0 sum:", np.sum(nrn_0))
print("nrn_1 sum:", np.sum(nrn_1))
print("nrn_2 sum:", np.sum(nrn_2))
print("nrn_3 sum:", np.sum(nrn_3))
print("nrn_4 sum:", np.sum(nrn_4))
print("nrn_5 sum:", np.sum(nrn_5))
print("nrn_6 sum:", np.sum(nrn_6))
print("nrn_7 sum:", np.sum(nrn_7))
print("nrn_8 sum:", np.sum(nrn_8))
print("nrn_9 sum:", np.sum(nrn_9))

In [ ]:
## Raster plot of snntorch versus fenrir
dataset = test_loader.dataset
data_tmp, target_tmp = dataset[6]

spike_data = spikegen.latency(data_tmp, num_steps=100, tau=5, threshold=0.01, linear=True, normalize=True, clip=True)
spike_data = spike_data.to(device)
spk_rec, mem_rec = net(spike_data)
pred = torch.argmax(spk_rec.sum(dim=0).squeeze()).item()

nrn_arr = np.stack([nrn_0, nrn_1, nrn_2, nrn_3, nrn_4, nrn_5, nrn_6, nrn_7, nrn_8, nrn_9], axis=1)
pred_nrn = np.argmax(nrn_arr.sum(axis=0))

fig, ax = plt.subplots(2, 1, figsize=(12, 7))
splt.raster(spk_rec[:, 0].view(100, -1), ax[0], s=25, c="black")
splt.raster(torch.from_numpy(nrn_arr).view(100, -1), ax[1], s=25, c="black")
ax[0].set_title(f"snntorch pred: {pred}, Target: {target_tmp}")
ax[1].set_title(f"fenrir pred: {pred_nrn}, Target: {target_tmp}")

In [ ]:
## Export the chosen datapoint for fenrir
exp_data = spike_data[:, 0, :, :]
exp_data = exp_data.view(exp_data.shape[0], -1)

debug = False
events = []

## Loop timesteps
for t in range(0, 100):
    t_data = exp_data[t, :]
    non_zero_indices = (t_data != 0).nonzero(as_tuple=True)[0]
    non_zero_values = t_data[non_zero_indices]
    
    if debug:
        for idx, val in zip(non_zero_indices.tolist(), non_zero_values.tolist()):
            print(f"Timestep {t}: index {idx} -> value {val}")
    
    for idx in non_zero_indices.tolist():
        events.append(idx)

binary_events = [format(idx, '010b') for idx in events]

with open('C:/home/university/8-semester/fenrir/src/design_sources/data/spike_data.txt', 'w') as f:
    for b in binary_events:
        f.write("0000000000000000000000" + b + '\n')